# **Python Database access and manipulation for shelter database**
*Author Het Thakkar*

In [ ]:
import pandas as pd

# !conda install psycopg2
# !pip install psycopg2
import numpy as np
import psycopg2 as db
import csv

In [ ]:
database = 'YOUR_DATABASE_NAME'
user = 'YOUR_USER_NAME'
password = 'YOUR__PASSWORD'
host = '127.0.0.1'
port = '5432'

schema = 'YOUR_SCHEMA_NAME'

In [ ]:
conn = db.connect(database = database, user = user, password = password, host = host, port = port)

**Run the following cell if you have transaction issues**

In [ ]:
conn.rollback()

Get cursor object and set search path

In [ ]:
cur = conn.cursor()

cur.execute('Set search_path to ' + schema)


Get number of tables in schema

In [ ]:
cur.execute('SELECT count(*) FROM pg_catalog.pg_tables where schemaname=\''+schema+'\';')

ntables = int(cur.fetchall()[0][0])

print(ntables)

In [ ]:
cur.execute('SELECT tablename FROM pg_catalog.pg_tables where schemaname=\''+schema+'\';')
tables = cur.fetchall()



Delete all rows in all tables


In [ ]:
print(tables)

for i in range(ntables):
    cur.execute('truncate ' + tables[i][0] + ' cascade')
    
    
conn.commit()

Download all current tables as separate csv files

In [ ]:
current_dir = !pwd

for i in range(ntables):
    with open(current_dir[0] +'\\DownloadFiles\\'+tables[i][0]+'.csv', 'w') as f:
        cur.copy_expert("COPY "+tables[i][0]+" TO STDOUT WITH CSV HEADER", f)
        
conn.commit() 

Insert into tables from CSV Files. NOTE: All csv files must be present in WD/CSV_Files with csv files corresponding to each table

In [ ]:
for i in range(ntables):
    with open('CSV_Files/' + tables[i][0] + '.csv', 'r') as f:
        reader = csv.reader(f)
        next(reader)  # Skip the header row.
        for row in reader:
            str = '(\'' +'\',\''.join(row) + '\')'
            cur.execute("INSERT INTO "+ tables[i][0] +" VALUES " + str)
        
conn.commit() 

Select statement example

In [ ]:
cur.execute('SELECT * FROM ' + tables[0][0])
try:
    rows = cur.fetchall()
    print(rows)
except:
    print('Prolly nothin in selected')

If you want to import it to a dataframe

In [ ]:
df = pd.read_sql_query('select * from shelter', conn)

print(df)

In [ ]:
conn.commit()